In [195]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import time
import pickle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import datasets
from sklearn import metrics

from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

## Pembacaan Dataset

In [196]:
data_test = pd.read_csv("tubes2_HeartDisease_test.csv")
data_train = pd.read_csv("tubes2_HeartDisease_train.csv")

In [197]:
data_train.head()

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,Column11,Column12,Column13,Column14
0,54,1,4,125,216,0,0,140,0,0,?,?,?,1
1,55,1,4,158,217,0,0,110,1,2.5,2,?,?,1
2,54,0,3,135,304,1,0,170,0,0,1,0,3,0
3,48,0,3,120,195,0,0,125,0,0,?,?,?,0
4,50,1,4,120,0,0,1,156,1,0,1,?,6,3


In [198]:
data_test.head()

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,Column11,Column12,Column13
0,60,1,2,160,267,1,1,157,0,0.5,2,?,?
1,61,1,4,148,203,0,0,161,0,0,1,1,7
2,54,1,4,130,242,0,0,91,1,1,2,?,?
3,48,1,4,120,260,0,0,115,0,2,2,?,?
4,57,0,1,130,308,0,0,98,0,1,2,?,?


In [199]:
print("Banyak Data Train :", len(data_train), "\nBanyak Data Test :", len(data_test))

Banyak Data Train : 779 
Banyak Data Test : 141


## 1. Pengolahan Data

### Penanganan NaN

#### Menghitung Jumlah NaN per Kolom

In [200]:
def countNanCol(df, colName) :
    count = 0
    for index, row in df.iterrows():
        if (row[colName] == "?") :
            count += 1
    return count

print("------------------- Data NaN pada data train -------------------")
for column in data_train:
    banyakKosong = countNanCol(data_train, column)
    print(column, "berisi \"?\" sebanyak :", banyakKosong, "\tyaitu :", banyakKosong/len(data_train)*100, "%")
    
print()
print("------------------- Data NaN pada data test -------------------")
for column in data_test:
    banyakKosong = countNanCol(data_test, column)
    print(column, "berisi \"?\" sebanyak :", banyakKosong, "\tyaitu :", banyakKosong/len(data_test)*100, "%")

------------------- Data NaN pada data train -------------------
Column1 berisi "?" sebanyak : 0 	yaitu : 0.0 %
Column2 berisi "?" sebanyak : 0 	yaitu : 0.0 %
Column3 berisi "?" sebanyak : 0 	yaitu : 0.0 %
Column4 berisi "?" sebanyak : 47 	yaitu : 6.033376123234917 %
Column5 berisi "?" sebanyak : 24 	yaitu : 3.080872913992298 %
Column6 berisi "?" sebanyak : 78 	yaitu : 10.012836970474968 %
Column7 berisi "?" sebanyak : 1 	yaitu : 0.12836970474967907 %
Column8 berisi "?" sebanyak : 44 	yaitu : 5.648267008985879 %
Column9 berisi "?" sebanyak : 44 	yaitu : 5.648267008985879 %
Column10 berisi "?" sebanyak : 49 	yaitu : 6.290115532734275 %
Column11 berisi "?" sebanyak : 262 	yaitu : 33.632862644415916 %
Column12 berisi "?" sebanyak : 514 	yaitu : 65.98202824133504 %
Column13 berisi "?" sebanyak : 408 	yaitu : 52.374839537869065 %
Column14 berisi "?" sebanyak : 0 	yaitu : 0.0 %

------------------- Data NaN pada data test -------------------
Column1 berisi "?" sebanyak : 0 	yaitu : 0.0 %
Col

#### Menghitung Jumlah NaN per Baris

In [201]:
def countNanRow(df, rowNum) :
    count = 0
    for column in df :
        if (df.loc[rowNum][column] == "?") :
            count += 1
    return count

print("------------------- Data NaN pada data train -------------------")
rowNan = [0 for i in range (13)]
# counting
for index, row in data_train.iterrows():
    rowNan[countNanRow(data_train, index)] += 1
for i in range (13):
    print("Kosong sebanyak", i, "ada sejumlah :", rowNan[i], "\tyakni :", rowNan[i]/len(data_train)*100, "%")
    
print()
print("------------------- Data NaN pada data test -------------------")
rowNan = [0 for i in range (13)]
# counting
for index, row in data_test.iterrows():
    rowNan[countNanRow(data_test, index)] += 1
for i in range (13):
    print("Kosong sebanyak", i, "ada sejumlah :", rowNan[i], "\tyakni :", rowNan[i]/len(data_train)*100, "%")

------------------- Data NaN pada data train -------------------
Kosong sebanyak 0 ada sejumlah : 256 	yakni : 32.86264441591784 %
Kosong sebanyak 1 ada sejumlah : 62 	yakni : 7.958921694480104 %
Kosong sebanyak 2 ada sejumlah : 172 	yakni : 22.0795892169448 %
Kosong sebanyak 3 ada sejumlah : 223 	yakni : 28.626444159178433 %
Kosong sebanyak 4 ada sejumlah : 21 	yakni : 2.6957637997432604 %
Kosong sebanyak 5 ada sejumlah : 2 	yakni : 0.25673940949935814 %
Kosong sebanyak 6 ada sejumlah : 1 	yakni : 0.12836970474967907 %
Kosong sebanyak 7 ada sejumlah : 40 	yakni : 5.134788189987163 %
Kosong sebanyak 8 ada sejumlah : 2 	yakni : 0.25673940949935814 %
Kosong sebanyak 9 ada sejumlah : 0 	yakni : 0.0 %
Kosong sebanyak 10 ada sejumlah : 0 	yakni : 0.0 %
Kosong sebanyak 11 ada sejumlah : 0 	yakni : 0.0 %
Kosong sebanyak 12 ada sejumlah : 0 	yakni : 0.0 %

------------------- Data NaN pada data test -------------------
Kosong sebanyak 0 ada sejumlah : 43 	yakni : 5.5198973042362 %
Kosong seban

#### Mengabaikan data pada baris dengan jumlah kosong sebanyak Threshold

In [222]:
threshold = 7

listDel = []
for index, row in data_train.iterrows():
    if (countNanRow(data_train, index) > threshold) :
        listDel.append(index)

data_train_clean = data_train.drop(listDel)
print("Jumlah data baru :", len(data_train_clean))

Jumlah data baru : 543


#### Mengisi data kosong dengan mean pada kolom 4, 5, 8, 10, 12 dan modus untuk sisanya

Mencari nilai data yang akan dimasukkan

In [203]:
## Cek data dulu pake mean atau median
replacer = []

def dropNanOnCol(df, colName) :
    listDel = []
    for index, row in df.iterrows():
        if (row[colName] == "?") :
            listDel.append(index)
    return df.drop(listDel)

changeWithMean = ['Column4', 'Column5', 'Column8', 'Column10', 'Column12']
for column in data_train_clean:
    cleanData = dropNanOnCol(data_train_clean, column)
    if (column in changeWithMean) :
        replacer.append(cleanData.median()[column])
#         replacer.append(np.asarray(cleanData.loc[:,column], dtype=np.float).mean())
    else :
        replacer.append(cleanData.mode()[column])
      
# special case
replacer[10] = replacer[10][0]

Mengisi nilai data

In [204]:
i = 0
for column in data_train_clean:
    for index, row in data_train_clean.iterrows():
        if (row[column] == "?") :
            if (type(replacer[i]) != str) : 
                data_train_clean.at[index, column] = replacer[i].item()
            else :
                data_train_clean.at[index, column] = replacer[i]
    i += 1


#### Mengabaikan kolom 12 dan 13

In [205]:
data_train_clean.drop(['Column12', 'Column13'],axis=1, inplace=True)

In [206]:
# use pd.concat to join the new columns with your original dataframe
data_train_ready = pd.concat([data_train_clean,pd.get_dummies(data_train_clean['Column3'], prefix='Column3')],axis=1)
data_train_ready = pd.concat([data_train_ready,pd.get_dummies(data_train_clean['Column7'], prefix='Column7')],axis=1)
data_train_ready = pd.concat([data_train_ready,pd.get_dummies(data_train_clean['Column11'], prefix='Column11')],axis=1)

#delete unused column
data_train_ready.drop(['Column3'],axis=1, inplace=True)
data_train_ready.drop(['Column7'],axis=1, inplace=True)
data_train_ready.drop(['Column11'],axis=1, inplace=True)

In [207]:
data_train_ready['Column14'].value_counts()

0    348
1    225
2     91
3     90
4     23
Name: Column14, dtype: int64

## 2. Eksperimen

#### Import modul terkait splitting data

In [208]:
from sklearn.model_selection import train_test_split

#### Fungsi penilaian

In [209]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 

def cal_accuracy(y_test, y_pred): 
      
    print("Confusion Matrix: \n", 
        confusion_matrix(y_test, y_pred)) 
      
    print ("Accuracy : \n", 
    accuracy_score(y_test,y_pred)*100) 
    print("Report : \n", 
    classification_report(y_test, y_pred)) 

#### Pemotongan data training dan data testing

In [210]:
data = data_train_ready.drop(columns=['Column14'])
target = data_train_ready['Column14']

data_train, data_test, target_train, target_test = train_test_split(data, target, test_size = 0.3, random_state = 100)

In [211]:
data_test.head()

,Column1,Column2,Column4,Column5,Column6,Column8,Column9,Column10,Column3_1,Column3_2,Column3_3,Column3_4,Column7_0,Column7_1,Column7_2,Column11_1,Column11_2,Column11_3
299,50,0,160,0,0,110,0,0,0,0,0,1,1,0,0,0,1,0
339,57,1,130,311,0,148,1,2,0,0,0,1,0,1,0,0,1,0
320,59,1,135,234,0,161,0,5,0,0,0,1,1,0,0,0,1,0
587,61,1,140,207,0,138,1,19,0,0,0,1,0,0,1,1,0,0
689,58,1,140,211,1,165,0,0,0,0,1,0,0,0,1,1,0,0


### Naive Bayes

In [212]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

model_gnb = gnb.fit(data_train, target_train)
y_pred = model_gnb.predict(data_test)

cal_accuracy(target_test, y_pred)

Confusion Matrix: 
 [[75  3  0 23  8]
 [13 11  0 41  7]
 [ 2  1  0 16  4]
 [ 2  1  0 17  4]
 [ 0  0  0  5  1]]
Accuracy : 
 44.44444444444444
Report : 
              precision    recall  f1-score   support

          0       0.82      0.69      0.75       109
          1       0.69      0.15      0.25        72
          2       0.00      0.00      0.00        23
          3       0.17      0.71      0.27        24
          4       0.04      0.17      0.07         6

avg / total       0.61      0.44      0.45       234



/home/hagairaja/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### kNN

In [218]:
from sklearn import neighbors
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.manifold import Isomap

n_neighbors = 15

# compressing all the features into two features
iso = Isomap(n_components=2)
projected = iso.fit_transform(data)
X = projected
y = target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 100)

h = .02  # step size in the mesh

model_knc = neighbors.KNeighborsClassifier(n_neighbors, weights='distance')
model_knc.fit(X_train, y_train)
y_pred = model_knc.predict(X_test)

cal_accuracy(y_test, y_pred)

Confusion Matrix: 
 [[25  4  1  3  0]
 [14 10  0  4  0]
 [ 1  2  4  0  0]
 [ 3  5  0  0  0]
 [ 0  1  1  0  0]]
Accuracy : 
 50.0
Report : 
              precision    recall  f1-score   support

          0       0.58      0.76      0.66        33
          1       0.45      0.36      0.40        28
          2       0.67      0.57      0.62         7
          3       0.00      0.00      0.00         8
          4       0.00      0.00      0.00         2

avg / total       0.47      0.50      0.48        78



/home/hagairaja/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Decision Tree

In [214]:
from sklearn import tree
dtc = tree.DecisionTreeClassifier(criterion="gini",min_samples_leaf=5)

model_dtc = dtc.fit(data_train, target_train)
y_pred = model_dtc.predict(data_test)

cal_accuracy(target_test, y_pred)

Confusion Matrix: 
 [[85 17  6  1  0]
 [25 31 11  5  0]
 [ 3 12  4  4  0]
 [ 8  8  6  2  0]
 [ 1  1  2  2  0]]
Accuracy : 
 52.13675213675214
Report : 
              precision    recall  f1-score   support

          0       0.70      0.78      0.74       109
          1       0.45      0.43      0.44        72
          2       0.14      0.17      0.15        23
          3       0.14      0.08      0.11        24
          4       0.00      0.00      0.00         6

avg / total       0.49      0.52      0.50       234



/home/hagairaja/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Neural Network MLP

In [215]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(activation='relu', hidden_layer_sizes=(120, 30), solver='lbfgs')

model_mlp = mlp.fit(data_train, target_train)
y_pred = model_mlp.predict(data_test)

cal_accuracy(target_test, y_pred)

Confusion Matrix: 
 [[97  6  3  3  0]
 [48 11  4  9  0]
 [ 8 10  1  4  0]
 [13  6  2  3  0]
 [ 1  1  2  2  0]]
Accuracy : 
 47.863247863247864
Report : 
              precision    recall  f1-score   support

          0       0.58      0.89      0.70       109
          1       0.32      0.15      0.21        72
          2       0.08      0.04      0.06        23
          3       0.14      0.12      0.13        24
          4       0.00      0.00      0.00         6

avg / total       0.39      0.48      0.41       234



/home/hagairaja/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## 3. Penyimpanan Model

In [226]:
#Menyimpan Naive Bayes Model
filename = 'nb_model.sav'
pickle.dump(gnb, open(filename, 'wb'))

#Menyimpan kNN Model
filename = 'knn_model.sav'
pickle.dump(model_knc, open(filename, 'wb'))

#Menyimpan Desicion Tree Mode
filename = 'dt_model.sav'
pickle.dump(dtc, open(filename, 'wb'))

#Menyimpan Neural Network MLP
filename = 'nnmlp_model.sav'
pickle.dump(mlp, open(filename, 'wb'))

## 4. Load Model

In [227]:
#Memuat Naive Bayes Model
filename = 'nb_model.sav'
NaiveBayesModel = pickle.load(open(filename, 'rb'))

#Memuat kNN Model
filename = 'knn_model.sav'
kNNModel = pickle.load(open(filename, 'rb'))

#Memuat Desicion Tree Model
filename = 'dt_model.sav'
DesicionTreeModel = pickle.load(open(filename, 'rb'))

#Memuat Neural Network MLP
filename = 'nnmlp_model.sav'
NeuralNetworkMLPModel = pickle.load(open(filename, 'rb'))